# Import backage

In [1]:
import sys
sys.path.append("../..")
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import random
%matplotlib inline

In [2]:
import torch
from torch import nn
from torch import distributions
from torch.nn.parameter import Parameter
from train import train
from models.models_new import Renorm_Dynamic
from dynamic_models_sis_new import Simple_Spring_Model
from ei.EI_calculation import EmergencePsi
from ei.EI_calculation import test_model_causal_multi_sis

from datetime import datetime
t0 = datetime.now()


use_cuda = torch.cuda.is_available()
device = torch.device('cuda:0') if use_cuda else torch.device('cpu')
device

device(type='cuda', index=0)

# Noise experiments

In [3]:
# nis+ CE macro
mul_batch_size = [0,5000,3000,1000]
sigma_list = [0.00001,0.0001,0.001,0.01,0.1,1]
experiments = len(sigma_list)
rou = -0.5
steps = 7
sz = 4
L = 1
mae2_w = 3
hidden_units = 64
T_total = 30001
dt = 0.01
spring = Simple_Spring_Model(device=device)

In [ ]:
# nis+ macro
sigma_list = [0.0001,0.001,0.01,0.1,1]
experiments = len(sigma_list)
scale = 2
eis_macro_e = np.zeros([experiments,T_total//500+1])
losses_macro_e = np.zeros([experiments,T_total//500+1])
MAEs_mstep_macro_e = np.zeros([experiments,T_total//500+1])
for e in range(experiments):
    sigma = sigma_list[e]
    seed = 1 
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    spring = Simple_Spring_Model(device=device)
    test_data = spring.generate_multistep_sir(size_list=[500,500], steps=10, sigma=sigma, rou=rou,lam=1,miu=0.5,dt=0.01) #sir
    train_data = spring.generate_multistep_sir(size_list=mul_batch_size, steps=steps, sigma=sigma,rou=rou,lam=1,miu=0.5,dt=0.01)
    
    eis_nisp, term1s_nisp, term2s_nisp, losses_nisp, MAEs_mstep_nisp, net = train(train_data=train_data, test_data=test_data, 
                                                                    sigma=sigma, rou=rou, sz=sz, scale=scale, L=L, 
                                                                    mae2_w=mae2_w, T2=T_total,framework='nis+')
    eis_macro_e[e,:] = eis_nisp
    losses_macro_e[e,:] = losses_nisp
    MAEs_mstep_macro_e[e,:] = MAEs_mstep_nisp
    torch.save(net.state_dict(), 'mdl_data/macro noise = '+str(sigma)+'.mdl')

check point-------o_0-> 16:20:36.330487; lasting 0:07:19.235179 seconds
Epoch: 0
Train loss: 0.4531
dEI: 1.0679
term1: 0.7085
term2: 1.4272
Test multistep loss: 0.4539
------------------------------------------------------------------------------------------------------------------------
check point-------o_1-> 16:20:39.260315; lasting 0:00:02.929828 seconds
check point-------o_0-> 16:20:49.345151; lasting 0:00:10.084836 seconds
Epoch: 500
Train loss: 0.0863
dEI: 1.7729
term1: 1.9331
term2: 1.6126
Test multistep loss: 0.0958
------------------------------------------------------------------------------------------------------------------------
check point-------o_1-> 16:20:53.990575; lasting 0:00:04.645424 seconds
check point-------w_0-> 16:21:04.059199; lasting 0:00:10.068624 seconds
check point-------w_1-> 16:21:04.070434; lasting 0:00:00.011235 seconds
check point-------w_2-> 16:25:45.518107; lasting 0:04:41.447673 seconds
check point-------w_3-> 16:25:45.572485; lasting 0:00:00.054

# Plot

In [9]:
ce_list=[]
macro_list=[]
micro_list=[]
psi_list=[]
MAE_raw = torch.nn.L1Loss(reduction='none')
for sigma in sigma_list:
    spring_data = spring.generate_multistep_sir(size_list=[500,500], steps=10, sigma=sigma, rou=rou,lam=1,miu=0.5,dt=dt) #sir
    s,sp,l,lp = spring_data
    net4 = Renorm_Dynamic(sym_size = sz, latent_size = 4, effect_size = sz, 
                         hidden_units = hidden_units, normalized_state=True, device = device)
    net2 = Renorm_Dynamic(sym_size = sz, latent_size = 2, effect_size = sz, 
                         hidden_units = hidden_units, normalized_state=True, device = device)
    net4.to(device=device)
    net2.to(device=device)
    net4.load_state_dict(torch.load('mdl_data/micro noise = '+str(sigma)+'.mdl'))
    net2.load_state_dict(torch.load('mdl_data/macro noise = '+str(sigma)+'.mdl'))
    print(sigma)
    _, _, V = net2.forward(s)
    psi, _, _ = EmergencePsi(s.cpu().numpy(), V.cpu().detach().numpy()) 
    ei4, sigmas0,weightsnet=test_model_causal_multi_sis(spring_data,MAE_raw,net4,sigma,4, L=L,num_samples = 1000)
    ei2, sigmas0,weightsnet=test_model_causal_multi_sis(spring_data,MAE_raw,net2,sigma,2, L=L,num_samples = 1000)
    ce_list.append(ei2[0]-ei4[0])
    macro_list.append(ei2[0])
    micro_list.append(ei4[0])
    psi_list.append(psi)




plt.figure(dpi=300)
color_label = ['#F4F1DE', '#DF7A5E', '#3C405B', '#82B29A', '#F2CC8E', '#A8DADC', '#457B9D', '#1D3557']
plt.plot(sigma_list, ce_list,label = r'$\Delta \mathcal{J}$', color=color_label[1])
plt.plot(sigma_list, macro_list,label = r'$macro \mathcal{J}$', color=color_label[2])
plt.plot(sigma_list, micro_list,label = r'$micro \mathcal{J}$', color=color_label[3])
plt.plot(sigma_list, psi_list,label = r'$\Psi$', color=color_label[4])
plt.legend()
plt.xlabel("sigma")
plt.xscale('log')
#plt.ylabel(r'$\mathcal{J}$')
plt.show()

1e-05


RuntimeError: Error(s) in loading state_dict for Renorm_Dynamic:
	size mismatch for dynamics.0.weight: copying a param with shape torch.Size([64, 4]) from checkpoint, the shape in current model is torch.Size([64, 2]).
	size mismatch for dynamics.4.weight: copying a param with shape torch.Size([4, 64]) from checkpoint, the shape in current model is torch.Size([2, 64]).
	size mismatch for dynamics.4.bias: copying a param with shape torch.Size([4]) from checkpoint, the shape in current model is torch.Size([2]).
	size mismatch for inv_dynamics.0.weight: copying a param with shape torch.Size([64, 4]) from checkpoint, the shape in current model is torch.Size([64, 2]).
	size mismatch for inv_dynamics.4.weight: copying a param with shape torch.Size([4, 64]) from checkpoint, the shape in current model is torch.Size([2, 64]).
	size mismatch for inv_dynamics.4.bias: copying a param with shape torch.Size([4]) from checkpoint, the shape in current model is torch.Size([2]).

In [ ]:
# nis+ micro
scale = 4
eis_micro_e = np.zeros([experiments,T_total//500+1])
losses_micro_e = np.zeros([experiments,T_total//500+1])
MAEs_mstep_micro_e = np.zeros([experiments,T_total//500+1])

for e in range(experiments):
    sigma = sigma_list[e]
    seed = 1 
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    spring = Simple_Spring_Model(device=device)
    test_data = spring.generate_multistep_sir(size_list=[500,500], steps=10, sigma=sigma, rou=rou,lam=1,miu=0.5,dt=0.01) #sir
    train_data = spring.generate_multistep_sir(size_list=mul_batch_size, steps=steps, sigma=sigma,rou=rou,lam=1,miu=0.5,dt=0.01)
    
    eis_nisp, term1s_nisp, term2s_nisp, losses_nisp, MAEs_mstep_nisp, net = train(train_data=train_data, test_data=test_data, 
                                                                    sigma=sigma, rou=rou, sz=sz, scale=scale, L=L, 
                                                                    mae2_w=mae2_w, T2=T_total,framework='nis+')
    eis_micro_e[e,:] = eis_nisp
    losses_micro_e[e,:] = losses_nisp
    MAEs_mstep_micro_e[e,:] = MAEs_mstep_nisp
    torch.save(net.state_dict(), 'mdl_data/micro noise = '+str(sigma)+'.mdl')